In [1]:
from tqdm import tqdm
import requests
import zipfile
import os
from io import BytesIO
import pandas as pd
from tabulate import tabulate
import chardet

In [2]:
extract_dir = './data/raw'
years = range(2010, 2026)
months = [f"{i:02d}" for i in range(1, 13)]
pro_data = './data/processed'

In [52]:
def download_file(url, out_path):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()

    
        with open(out_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
            
        print(f"Plik pomyślnie pobrany i zapisany jako: {out_path}")
        return True
    except requests.exceptions.RequestException as e:
        print(f"\n❌ Błąd podczas pobierania: {e}")
        return False
    except Exception as e:
        print(f"\n❌ Wystąpił nieoczekiwany błąd: {e}")
        return False
        
def handle_zip(zip_path, rm_zip=True):
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            extract_path = os.path.splitext(zip_path)[0]
            zip_ref.extractall(extract_path)
            print(f"Pliki zostały rozpakowane do: {extract_path}")
        if rm_zip:
            os.remove(zip_path)
            print(f"Plik ZIP został usunięty: {zip_path}")
        return True
    except zipfile.BadZipFile:
        print("❌ Błąd: Plik nie jest prawidłowym archiwum ZIP.")
        return False
    except Exception as e:
        print(f"\n❌ Wystąpił nieoczekiwany błąd podczas rozpakowywania: {e}")
        return False
        

In [53]:
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir, exist_ok=True)


    for y in years:
        for m in months:
            uri = f"https://danepubliczne.imgw.pl/pl/datastore/getfiledown/Arch/Telemetria/Meteo/{y}/Meteo_{y}-{m}.zip"
            filename = uri.split('/')[-1]
            out_path = os.path.join(extract_dir, filename)
            if download_file(uri, out_path):

                if handle_zip(out_path, rm_zip=True) == False:
                    uri = f"https://danepubliczne.imgw.pl/pl/datastore/getfiledown/Arch/Telemetria/Meteo/{y}/Meteo_{y}-{m}.ZIP"
                    out_path = os.path.join(extract_dir, filename+"_2")
                    if download_file(uri, out_path):
                        handle_zip(out_path, rm_zip=True) == False
                print(f"Pobrano i rozpakowano dane dla {y}-{m}")

Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-01.zip
❌ Błąd: Plik nie jest prawidłowym archiwum ZIP.
Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-01.zip_2
Pliki zostały rozpakowane do: ./data/raw\Meteo_2010-01
Plik ZIP został usunięty: ./data/raw\Meteo_2010-01.zip_2
Pobrano i rozpakowano dane dla 2010-01
Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-02.zip
❌ Błąd: Plik nie jest prawidłowym archiwum ZIP.
Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-02.zip_2
Pliki zostały rozpakowane do: ./data/raw\Meteo_2010-02
Plik ZIP został usunięty: ./data/raw\Meteo_2010-02.zip_2
Pobrano i rozpakowano dane dla 2010-02
Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-03.zip
❌ Błąd: Plik nie jest prawidłowym archiwum ZIP.
Plik pomyślnie pobrany i zapisany jako: ./data/raw\Meteo_2010-03.zip_2
Pliki zostały rozpakowane do: ./data/raw\Meteo_2010-03
Plik ZIP został usunięty: ./data/raw\Meteo_2010-03.zip_2
Pobrano i rozpakowano 

In [56]:
paths = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith('.zip'):
            os.remove(os.path.join(root, file))

In [4]:
os.makedirs(extract_dir, exist_ok=True)

paths = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.endswith('.csv'):
            full_path = os.path.join(root, file)
            paths.append(full_path)
            
print(f"\nZnaleziono {len(paths)} plików CSV.")


Znaleziono 2041 plików CSV.


In [5]:
headers = ["Station ID", "Param", "Datetime", "Value"]
KOD_WROCLAW_STRACHOWICE = 351160424
KOD_WROCLAW = 351160425

raw_dfs=[]
for file in tqdm(paths):
    df = pd.read_csv(file, sep=';', header=None, names=headers, encoding='UTF-8', index_col=False, low_memory=False)
    df = df[(df['Station ID'] == KOD_WROCLAW_STRACHOWICE) | (df['Station ID'] == KOD_WROCLAW)]
    raw_dfs.append(df)
print(tabulate(raw_dfs[-1].head(), headers='keys', tablefmt='psql'))

 61%|██████    | 1244/2041 [07:55<05:04,  2.62it/s]


KeyboardInterrupt: 

In [ ]:
all_data = pd.concat(raw_dfs, ignore_index=True)
all_data.reset_index(drop=True, inplace=True)

del raw_dfs

print(f"Rozmiar połączonego DataFrame: {all_data.shape}")
print(tabulate(all_data.head(), headers='keys', tablefmt='psql'))

Rozmiar połączonego DataFrame: (228088, 4)
+----+--------------+---------+------------------+---------+
|    |   Station ID | Param   | Datetime         |   Value |
|----+--------------+---------+------------------+---------|
|  0 |    351160424 | B00202A | 2024-01-01 00:00 |     279 |
|  1 |    351160424 | B00202A | 2024-01-01 00:10 |     252 |
|  2 |    351160424 | B00202A | 2024-01-01 00:20 |     253 |
|  3 |    351160424 | B00202A | 2024-01-01 00:30 |     276 |
|  4 |    351160424 | B00202A | 2024-01-01 00:40 |     128 |
+----+--------------+---------+------------------+---------+


In [ ]:
all_data['Value'] = pd.to_numeric(all_data['Value'], errors='coerce')
all_data_pivot = all_data.pivot_table(index=['Station ID', 'Datetime'], columns='Param', values='Value').reset_index()
print(f"Rozmiar DataFrame po pivotowaniu: {all_data_pivot.shape}")
print(tabulate(all_data_pivot.describe(), headers='keys', tablefmt='psql'))

Rozmiar DataFrame po pivotowaniu: (34272, 13)
+-------+----------------+------------+-------------+-------------+------------+---------------+----------------+-------------+-------------+-----------+------------+-----------+
|       |     Station ID |    B00202A |     B00300S |     B00305A |    B00604S |       B00606S |        B00608S |     B00702A |     B00703A |   B00714A |    B00802A |   B00910A |
|-------+----------------+------------+-------------+-------------+------------+---------------+----------------+-------------+-------------+-----------+------------+-----------|
| count | 34272          | 34040      | 18716       | 18729       | 168        | 5332          | 16990          | 18472       | 18655       |   201     | 34272      |  12       |
| mean  |     3.5116e+08 |   205.444  |     2.07325 |     1.26927 |   0.252976 |    0.00834584 |     0.00208946 |     2.71159 |     4.57707 |    13.199 |    80.7146 |   8.25    |
| std   |     0.500007   |    80.5537 |     4.3473  |     4

In [ ]:
params_dic = {
    "B00300S": "Temperatura powietrza (oficjalna)",
    "B00305A": "Temperatura gruntu (czujnik)",
    "B00202A": "Kierunek wiatru (czujnik)",
    "B00702A": "Średnia prędkość wiatru czujnik 10 minut",
    "B00703A": "Prędkość maksymalna (czujnik)",
    "B00608S": "Suma opadu 10 minutowego",
    "B00604S": "Suma opadu dobowego",
    "B00606S": "Suma opadu godzinowego",
    "B00802A": "Wilgotność względna powietrza (czujnik)",
    "B00714A": "Największy poryw w okresie 10min ze stacji Synoptycznej",
    "B00910A": "Zapas wody w śniegu (obserwator)"
}

all_data_pivot.rename(columns=params_dic, inplace=True)
all_data_pivot['Datetime'] = pd.to_datetime(all_data_pivot['Datetime'], format='%Y-%m-%d %H:%M')
all_data_pivot.sort_values(by=['Station ID', 'Datetime'], inplace=True)
all_data_pivot.reset_index(drop=True, inplace=True)

print("OPIS")
print(tabulate(all_data_pivot.describe(), headers='keys', tablefmt='psql'))

print("GŁOWA")
print(tabulate(all_data_pivot.head(), headers='keys', tablefmt='psql'))

+----+--------------+---------------------+-----------------------------+-------------------------------------+--------------------------------+-----------------------+--------------------------+----------------------------+--------------------------------------------+---------------------------------+-----------------------------------------------------------+-------------------------------------------+------------------------------------+
|    |   Station ID | Datetime            |   Kierunek wiatru (czujnik) |   Temperatura powietrza (oficjalna) |   Temperatura gruntu (czujnik) |   Suma opadu dobowego |   Suma opadu godzinowego |   Suma opadu 10 minutowego |   Średnia prędkość wiatru czujnik 10 minut |   Prędkość maksymalna (czujnik) |   Największy poryw w okresie 10min ze stacji Synoptycznej |   Wilgotność względna powietrza (czujnik) |   Zapas wody w śniegu (obserwator) |
|----+--------------+---------------------+-----------------------------+-------------------------------------